In [1]:
import glob

In [5]:
submission_folder='submissions/'

In [7]:
for submission_dir in glob.glob('%s/*' % submission_folder):
    print(submission_dir)

submissions/paramitamirza
submissions/superlyc
submissions/CarlaAbreu
submissions/Piek
submissions/ID-DE
submissions/baseline1
